## Data Preparation for Sierra Leone
This notebook focuses on preparing the wind and solar dataset for analysis.

The goal is to clean, structure, and validate the dataset so that it’s ready for exploration and visualization.

It follows the assignment requirements step by step: loading data, summary statistics, missing value report, outlier detection and flagging, data cleaning and imputation, and saving the cleaned dataset.

All steps use the reusable functions defined in ```data_preparation.py```, ensuring consistency and reproducibility across multiple datasets.

## Imports

In [1]:
import sys
import pandas as pd;

sys.path.append('../../../scripts')
from data_cleaning import (
    load_data,
    get_summary_report,
    calculate_zscore_and_flag_outliers,
    clean_and_impute,
    save_cleaned_data
)

## Load Raw Data

In [2]:
raw_file_path = "../../../data/sierra-leone/sierraleone-bumbuna.csv"
output_dir = "../../../data/sierra-leone/"
df = load_data(raw_file_path) 

✅ 'Timestamp' column successfully converted to datetime objects.


## Observe the Data

In [3]:
print("## 📊 Head: First 5 Rows for Visual Check\n")
df.head()

## 📊 Head: First 5 Rows for Visual Check



,Timestamp,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
0,2021-10-30 00:01:00,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.1,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6,NaN
1,2021-10-30 00:02:00,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.2,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6,NaN
2,2021-10-30 00:03:00,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.2,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6,NaN
3,2021-10-30 00:04:00,-0.7,0.0,-0.8,0.0,0.0,21.9,99.3,0.0,0.0,0.0,0.0,0.0,1002,0,0.1,22.3,22.6,NaN
4,2021-10-30 00:05:00,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.3,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6,NaN


In [4]:
print("## ℹ️ Info: Data Types, Non-Null Counts, and Memory Usage\n")
df.info()

## ℹ️ Info: Data Types, Non-Null Counts, and Memory Usage

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525600 entries, 0 to 525599
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Timestamp      525600 non-null  datetime64[ns]
 1   GHI            525600 non-null  float64       
 2   DNI            525600 non-null  float64       
 3   DHI            525600 non-null  float64       
 4   ModA           525600 non-null  float64       
 5   ModB           525600 non-null  float64       
 6   Tamb           525600 non-null  float64       
 7   RH             525600 non-null  float64       
 8   WS             525600 non-null  float64       
 9   WSgust         525600 non-null  float64       
 10  WSstdev        525600 non-null  float64       
 11  WD             525600 non-null  float64       
 12  WDstdev        525600 non-null  float64       
 13  BP             525600 non-null  int64        

The raw dataset includes columns like GHI, DNI, DHI, ModA, ModB, WS, WSgust, WD, Tamb, RH, and Timestamp. Some columns may have missing values and potential outliers.

## Summary Statistics & Missing-Value Report

In [5]:
get_summary_report(df)


--- 1. Summary Statistics ---
                           Timestamp            GHI            DNI  \
count                         525600  525600.000000  525600.000000   
mean   2022-04-30 12:00:30.000000768     201.957515     116.376337   
min              2021-10-30 00:01:00     -19.500000      -7.800000   
25%              2022-01-29 06:00:45      -2.800000      -0.300000   
50%              2022-04-30 12:00:30       0.300000      -0.100000   
75%              2022-07-30 18:00:15     362.400000     107.000000   
max              2022-10-30 00:00:00    1499.000000     946.000000   
std                              NaN     298.495150     218.652659   

                 DHI           ModA           ModB           Tamb  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      113.720571     206.643095     198.114691      26.319394   
min       -17.900000       0.000000       0.000000      12.300000   
25%        -3.800000       0.000000       0.000000      23.100

We can see which columns have high missing values (>5%) and examine summary statistics for numeric columns. This identifies potential cleaning needs.

The Comments column contained 100% missing values, providing no useful information for analysis. Therefore, it was dropped to clean the dataset and simplify further processing.

In [6]:
df = df.drop(['Comments'], axis= 1).copy()
    
print("✅ 'Comments' column dropped due to 100% missing values.")
print("-" * 50)
print("After Dropping 'Comments' Column:")
df.head()

✅ 'Comments' column dropped due to 100% missing values.
--------------------------------------------------
After Dropping 'Comments' Column:


,Timestamp,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB
0,2021-10-30 00:01:00,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.1,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6
1,2021-10-30 00:02:00,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.2,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6
2,2021-10-30 00:03:00,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.2,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6
3,2021-10-30 00:04:00,-0.7,0.0,-0.8,0.0,0.0,21.9,99.3,0.0,0.0,0.0,0.0,0.0,1002,0,0.1,22.3,22.6
4,2021-10-30 00:05:00,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.3,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6


Negative solar radiation readings correspond to nighttime hours (when the sun isn’t shining). These values are physically meaningless for solar analysis, so they’re excluded to focus on daytime data only.

In [7]:
# Remove negative solar radiation values (GHI, DNI, DHI)
solar_cols = ['GHI', 'DNI', 'DHI']

# Count how many negative values exist before removal
negative_counts = (df[solar_cols] < 0).sum()
print("Negative value count before removal:\n", negative_counts)

# Remove rows where any solar column is negative
df = df[(df[solar_cols] >= 0).all(axis=1)]

# Reset index after filtering
df = df.reset_index(drop=True)

print(f"\nData shape after removing negatives: {df.shape}")


Negative value count before removal:
 GHI    261135
DNI    266352
DHI    263128
dtype: int64

Data shape after removing negatives: (238872, 18)


## Outlier Detection

In [8]:
df = calculate_zscore_and_flag_outliers(df)

outlier_rows = df[df['Outliers_Flag'] == True]
print("Count of rows with outlier values - ", df['Outliers_Flag'].sum())
print(outlier_rows.head(5))


🔹 Calculating Z-scores and flagging outliers for the following columns:
   ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
✅ Outliers flagged in 1760 rows.
Count of rows with outlier values -  1760
               Timestamp   GHI  DNI   DHI  ModA  ModB  Tamb    RH   WS  \
1265 2021-10-31 17:36:00  35.2  3.6  34.3  35.1  33.1  31.2  66.8  7.5   
1266 2021-10-31 17:37:00  28.9  0.3  29.0  30.8  29.0  30.9  66.7  6.2   
1267 2021-10-31 17:38:00  26.0  0.2  26.2  29.3  27.8  30.6  63.6  5.2   
1268 2021-10-31 17:39:00  24.6  0.2  24.8  28.9  27.4  30.2  63.6  5.3   
1269 2021-10-31 17:40:00  23.9  0.2  24.3  29.4  28.0  29.9  64.5  5.3   

      WSgust  WSstdev     WD  WDstdev   BP  Cleaning  Precipitation  TModA  \
1265     9.7      1.3  138.2     27.2  999         0            0.0   37.6   
1266     7.9      0.9  111.9     15.5  999         0            0.0   36.4   
1267     7.2      1.0   99.4     20.3  999         0            0.0   35.1   
1268     7.4      1.3  101.5     18.6  

## Cleaning & Imputation

In [9]:
columns_to_impute = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'Tamb', 'WS', 'WSgust', 'RH']
df_cleaned = clean_and_impute(df, columns_to_impute)

cleaned_file_path = output_dir + "sierra_leone_clean.csv"
save_cleaned_data(df_cleaned, cleaned_file_path) 

🔹 Replacing outliers with median for the following columns:
   ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'Tamb', 'WS', 'WSgust', 'RH']
🔹 Imputing remaining missing values with median for key columns.
✅ Outliers replaced and missing values imputed.

✅ Cleaned Data Saved successfully to: ../../../data/sierra-leone/sierra_leone_clean.csv
